In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skcriteria import Data, MIN, MAX

## Bring in data

In [ ]:
lst_path = './aqua95_mtCounties.geojson' 
census_path = '../../../../Data/Census/acs2017_county_data_MT.csv'
res_path = '../shapefiles/MontanaReservations_shp/MontanReservations_4326.shp'
kh_path = './killer-heat-data-by-county-100.csv'
kh_cols = ['State', 'County', 'Historical', 'MC_slow', 'MC_no', 'EC_slow', 'EC_no', 'Rapid_Action']
lst_gdf = gpd.read_file(lst_path)
cen_df = pd.read_csv(census_path)
res_bounds = gpd.read_file(res_path)
kh_df = pd.read_csv(kh_path, skiprows=3, names=kh_cols)
kh_df = kh_df[kh_df['State'] == 'MT']
kh_df.head()

## Prepare DataFrame for analysis

In [ ]:
lst_gdf['mean_F'] = (((lst_gdf['mean'] * 0.02) - 273.15) * (9/5.)) + 32
lst_gdf['GEOID'] = lst_gdf['GEOID'].astype(int)
kh_df['killer_heat_diff'] = kh_df['MC_no'] - kh_df['Historical']
cen_df = pd.merge(cen_df, kh_df[['County', 'killer_heat_diff']], how='inner', left_on='County', right_on='County')
gdf = pd.merge(lst_gdf, cen_df, how='inner', left_on='GEOID', right_on='CountyId')
cols = ['CountyId', 'mean_F', 'killer_heat_diff', 'Income', 'Poverty', 'ChildPoverty', 'Construction', 'Production', 'Unemployment', 'Professional']
data = gdf[cols]
data = data.set_index('CountyId')
data_norm = data/((data**2).sum())**0.5

## Rank socio-economic factors for Roosevelt

In [ ]:
df_serank = gdf[cols].rank(method='min')
df_serank['CountyId'] = gdf['CountyId']
df_serank['Name'] = gdf['NAME']
df_serank['Income'] = 57 - df_serank['Income']
df_serank['Professional'] = 57 - df_serank['Professional']
# df_serank[df_serank['Name'] == 'Roosevelt']
df_serank.to_csv('county_variable_rankings.csv')

## Create scikit object for analysis

In [ ]:
# crit = [MAX, MIN, MAX, MAX, MAX, MAX, MAX]
crit = [MIN, MIN, MAX, MIN, MIN, MIN, MIN, MIN, MAX]
counties = data.index
cnms = data.columns
mtx = data.values
wts = [0.255, 0.255, 0.07, 0.07, 0.07, 0.07, 0.07, 0.07, 0.07]
# wts = [0.93, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
mca_data = Data(mtx, crit, anames=counties, cnames=cnms, weights=wts)

## MCA

In [ ]:
from skcriteria.madm import closeness, simple
dm = closeness.TOPSIS()
dec = dm.decide(mca_data)
gdf['vulnerability'] = dec.e_.closeness
gdf['vul_rank'] = dec.rank_
# dec.rank_[dec.e_.points==max(dec.e_.points)]
# counties[dec.rank_==min(dec.rank_)], counties[dec.rank_==max(dec.rank_)]
# dec.e_.closeness[dec.rank_==max(dec.rank_)], min(dec.e_.closeness)

In [ ]:
gdf[['NAME', 'vulnerability']].to_csv('county_vulnerability_closeness.csv')

## Plot

In [ ]:
lst_gdf = lst_gdf.to_crs(epsg=3857)
ax = lst_gdf.plot(column="mean_F", cmap='OrRd', legend=True, figsize=[10,3], edgecolor='k', linewidth=0.5)
ax.set_title('Extreme Heat Land Surface Temeperature')
ax.axis('off')

In [ ]:
fig = ax.get_figure()
fig.savefig("modis_lst_95th_perc_mt_counties.png", bbox_inches="tight", dpi=300)

In [ ]:
ax = gdf.plot(column='vul_rank', cmap='OrRd', legend=True, figsize=[10, 3])
# res_bounds.geometry.boundary.plot(ax=ax, color=None, edgecolor='k', linewidth=1)  # Add reservation boundaries
ax.set_title('Ranked Vulnerability to Heat')
ax.axis('off')

In [ ]:
# fig = ax.get_figure()
# fig.savefig("ranked_heat_vulnerability_with_res.png", bbox_inches="tight", dpi=300)

## High, medium, low groupings

In [ ]:
# lower_perc = np.percentile(gdf.vulnerability, 33.33333)
# upper_perc = np.percentile(gdf.vulnerability, 66.66667)
lower_abs = 0.25
mid_abs = 0.5
upper_abs = 0.75
upper = upper_abs
mid = mid_abs
lower = lower_abs
gdf['rank_grouped'] = np.zeros(len(gdf))
gdf['rank_grouped'][gdf['vulnerability'] >= upper] = 1
gdf['rank_grouped'][(gdf['vulnerability'] < upper) & (gdf['vulnerability'] >= lower)] = 2
gdf['rank_grouped'][gdf['vulnerability'] < lower] = 3
grp_dict = {1.: 'Low', 2.: 'Medium', 3.: 'High'}


def replace_legend_items(legend, mapping):
    for txt in legend.texts:
        for k,v in mapping.items():
            if txt.get_text() == str(k):
                txt.set_text(v)
 

In [ ]:
gdf = gdf.to_crs(epsg=3857)
ax = gdf.plot(column='rank_grouped', cmap='OrRd', categorical=True, 
    legend=True, figsize=[11, 4], legend_kwds={'loc': 'lower left'}, edgecolor='k')
replace_legend_items(ax.get_legend(), grp_dict)
ax.set_title('Vulnerability to Heat')
ax.axis('off')

In [ ]:
fig = ax.get_figure()
fig.savefig("grouped_vulnerability_absolute.png", bbox_inches="tight", dpi=300)

In [ ]:
(gdf['rank_grouped'] == 2.0).sum()

In [ ]:
lower_abs = 0.25
mid_abs = 0.5
upper_abs = 0.75
upper = upper_abs
mid = mid_abs
lower = lower_abs
gdf['rank_grouped'] = np.zeros(len(gdf))
gdf['rank_grouped'][gdf['vulnerability'] >= upper] = 1
gdf['rank_grouped'][(gdf['vulnerability'] < upper) & (gdf['vulnerability'] >= mid)] = 2
gdf['rank_grouped'][(gdf['vulnerability'] < mid) & (gdf['vulnerability'] >= lower)] = 3
gdf['rank_grouped'][gdf['vulnerability'] < lower] = 4
grp_dict = {1.: 'Low', 2.: 'Medium-low', 3.: 'Medium-high', 4.:'High'}

In [ ]:
gdf = gdf.to_crs(epsg=3857)
ax = gdf.plot(column='rank_grouped', cmap='OrRd', categorical=True, 
    legend=True, figsize=[12, 6], legend_kwds={'loc': 'lower left'}, edgecolor='k')
replace_legend_items(ax.get_legend(), grp_dict)
ax.set_title('Vulnerability to Heat')
ax.axis('off')

In [ ]:
fig = ax.get_figure()
fig.savefig("grouped_vulnerability_absolute_4cats.png", bbox_inches="tight", dpi=300)

In [ ]:
(gdf['rank_grouped'] == 1).sum()

In [ ]:
26+1+12+17
